## DSL Job3

Grammar structure:

```
{'toks': set(token), 'vars': dict(var: definition), 'hvar': var}
token : (class, value)
class : int
value : str
var : str                 # non-terminal name
definition : list(rule)
rule : list(var | token)  # right side of the rule
```

**1. FIRST()**

`FIRST(X)` is a function that returns a set of terminal symbols that the strings derived from `X` begin with. We will implement this function in the following way: if the first symbol of the rule is a nonterminal, add it to the result set, else call the function recursively on the observed symbol.

In [1]:
def first(tokens, rules, symbol):
  result = set()

  for rule in rules[symbol]:
    if rule[0] in tokens:
      result.add(rule[0])
    else:
      result = result.union(first(tokens, rules, rule[0]))

  return result

**2. FOLLOW()**

`FOLLOW(X)` is a function that returns a set of terminal symbols that may appear sequentially after `X`. We will find all productions that contain `X` in them and take the symbol that follows `X`, lets call it `Y`. If the `X` is not the last symbol, we will find all first symbols of the strings derived from `Y` using the `FIRST(Y)` function. Else we will call the `FOLLOW()` function recursively on the symbol that string with `X` is derived from.

In [12]:
def follow(tokens, vars, symbol):
  result = set()
  
  for var, rules in vars.items():
    for rule in rules:
      indexes = [i for i, s in enumerate(rule) if s == symbol]
      
      for i in indexes:
        j = i + 1
        if j < len(rule):
          if rule[j] in tokens:
            result.add(rule[j])
          else:
            result = result.union(first(tokens, vars, rule[j]))
        elif var != symbol:
            result = result.union(follow(tokens, vars, var))

  return result

In [22]:
GRAMMAR = {
  'toks': set([
      ('type', 'a1'),
      ('type', 'a2'),
      ('type2', 'b1'),
      ('type3', 'c1'),
  ]),
  'vars': {
      'S': [
        ['A', ('type2', 'b1')],
        ['B', ('type3', 'c1')],
        [('type', 'a1'), ('type', 'a2')],
      ],
      'A': [
        ['B']
      ],
      'B': [
        [('type2', 'b1')]
      ],
      'E': [[]],
      'K': [['E']]
  },
  'hvar': 'S'
}

print(first(GRAMMAR['toks'], GRAMMAR['vars'], GRAMMAR['hvar']))

print(follow(GRAMMAR['toks'], GRAMMAR['vars'], 'S'))
print(follow(GRAMMAR['toks'], GRAMMAR['vars'], 'A'))
print(follow(GRAMMAR['toks'], GRAMMAR['vars'], 'B'))
print(follow(GRAMMAR['toks'], GRAMMAR['vars'], 'K'))

{('type2', 'b1'), ('type', 'a1')}
set()
{('type2', 'b1')}
{('type3', 'c1'), ('type2', 'b1')}
set()
